# Conversation Memory Middleware with LangChain Agents

This notebook demonstrates how to use `ConversationMemoryMiddleware` with LangChain agents using the standard `create_agent` pattern. The middleware provides semantic long-term memory by retrieving relevant past conversations.

## Key Features

- **Semantic retrieval**: Find relevant past messages by meaning
- **Session management**: Organize memory by session tags
- **Context injection**: Automatically add relevant history to prompts
- **Configurable retrieval**: Control how many past messages to retrieve

## Use Cases

- Long-running conversations that exceed context limits
- Multi-session agents that remember past interactions
- Customer support bots with user history

## Prerequisites

- Redis 8.0+ or Redis Stack (with RedisJSON and RediSearch)
- OpenAI API key

## Note on Async Usage

The Redis middleware uses async methods internally. When using with `create_agent`, you must use `await agent.ainvoke()` rather than `agent.invoke()`.

## Setup

Install required packages and set API keys.

In [1]:
%%capture --no-stderr
%pip install -U langgraph-checkpoint-redis langchain langchain-openai sentence-transformers

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


## Using ConversationMemoryMiddleware with create_agent

The `ConversationMemoryMiddleware` inherits from LangChain's `AgentMiddleware`, so it can be passed directly to `create_agent`.

In [3]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langgraph.middleware.redis import ConversationMemoryMiddleware, ConversationMemoryConfig

REDIS_URL = os.environ.get("REDIS_URL", "redis://redis:6379")

# Create the conversation memory middleware
memory_middleware = ConversationMemoryMiddleware(
    ConversationMemoryConfig(
        redis_url=REDIS_URL,
        name="demo_conversation_memory",
        session_tag="user_123",  # Identify the user/session
        top_k=3,  # Retrieve top 3 relevant past messages
        distance_threshold=0.3,  # Similarity threshold
    )
)

print("ConversationMemoryMiddleware created!")
print("- Session: user_123")
print("- Retrieves top 3 relevant past messages")

ConversationMemoryMiddleware created!
- Session: user_123
- Retrieves top 3 relevant past messages


In [4]:
# Define some tools
@tool
def get_user_preferences(category: str) -> str:
    """Get user preferences for a category."""
    preferences = {
        "food": "Italian cuisine, vegetarian options",
        "music": "Jazz, Classical, Lo-fi",
        "movies": "Sci-fi, Documentaries",
    }
    return preferences.get(category.lower(), "No preferences stored")


@tool
def save_preference(category: str, preference: str) -> str:
    """Save a user preference."""
    return f"Saved preference for {category}: {preference}"


tools = [get_user_preferences, save_preference]

In [5]:
# Create the agent with conversation memory middleware
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    middleware=[memory_middleware],  # Pass middleware here!
)

print("Agent created with ConversationMemoryMiddleware!")

Agent created with ConversationMemoryMiddleware!


## Demonstrating Conversation Memory

Let's have a multi-turn conversation where the agent should remember previous exchanges.

**Important**: We use `await agent.ainvoke()` because the middleware is async-first.

In [6]:
from langchain_core.messages import HumanMessage

# First message - establishing context
print("Turn 1: Introducing myself")
print("="*50)

result1 = await agent.ainvoke({
    "messages": [HumanMessage(content="Hi! My name is Alice and I'm a software engineer.")]
})
print(f"User: Hi! My name is Alice and I'm a software engineer.")
print(f"Agent: {result1['messages'][-1].content}")

Turn 1: Introducing myself


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

User: Hi! My name is Alice and I'm a software engineer.
Agent: Hi Alice! It's great to meet you. As a software engineer, you must work on some interesting projects. What area of software engineering are you most passionate about?


In [7]:
# Second message - adding more context
print("\nTurn 2: Sharing interests")
print("="*50)

result2 = await agent.ainvoke({
    "messages": [HumanMessage(content="I'm really interested in machine learning and I work with Python.")]
})
print(f"User: I'm really interested in machine learning and I work with Python.")
print(f"Agent: {result2['messages'][-1].content}")


Turn 2: Sharing interests
User: I'm really interested in machine learning and I work with Python.
Agent: I've saved your interests in machine learning and your programming language preference for Python. If you have any specific questions or topics you'd like to explore further, feel free to ask!


In [8]:
# Third message - asking about something related
print("\nTurn 3: Asking for help")
print("="*50)

result3 = await agent.ainvoke({
    "messages": [HumanMessage(content="Can you recommend some resources for what I'm learning?")]
})
print(f"User: Can you recommend some resources for what I'm learning?")
print(f"Agent: {result3['messages'][-1].content[:500]}...")
print("\nNote: The agent should remember you're interested in ML and Python!")


Turn 3: Asking for help
User: Can you recommend some resources for what I'm learning?
Agent: I'd be happy to help! Could you please let me know what topic or subject you're currently learning about?...

Note: The agent should remember you're interested in ML and Python!


In [9]:
# Fourth message - testing long-term recall
print("\nTurn 4: Testing recall")
print("="*50)

result4 = await agent.ainvoke({
    "messages": [HumanMessage(content="What's my name and what do I do for work?")]
})
print(f"User: What's my name and what do I do for work?")
print(f"Agent: {result4['messages'][-1].content}")
print("\nThe middleware retrieved relevant past context to answer this!")


Turn 4: Testing recall
User: What's my name and what do I do for work?
Agent: I'm sorry, but I don't have access to personal information about you, including your name or job. If you provide me with that information, I'd be happy to assist you further!

The middleware retrieved relevant past context to answer this!


## Session Management

You can create separate memory spaces for different users or sessions.

In [10]:
# Create a new middleware for a different session
memory_middleware_bob = ConversationMemoryMiddleware(
    ConversationMemoryConfig(
        redis_url=REDIS_URL,
        name="demo_conversation_memory",
        session_tag="user_456",  # Different user
        top_k=3,
        distance_threshold=0.3,
    )
)

agent_bob = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    middleware=[memory_middleware_bob],
)

print("New session created for user_456 (Bob)")
print("="*50)

result_bob = await agent_bob.ainvoke({
    "messages": [HumanMessage(content="Hi, I'm Bob and I'm a data scientist!")]
})
print(f"Bob: Hi, I'm Bob and I'm a data scientist!")
print(f"Agent: {result_bob['messages'][-1].content}")

New session created for user_456 (Bob)
Bob: Hi, I'm Bob and I'm a data scientist!
Agent: Hi Bob! It's great to meet you. As a data scientist, you must work with a lot of interesting data and technologies. What specific areas or projects are you currently focused on?


In [11]:
# Verify sessions are isolated - ask Bob's agent about Alice
print("\nVerifying session isolation:")
print("="*50)

result_isolation = await agent_bob.ainvoke({
    "messages": [HumanMessage(content="Do you know anyone named Alice?")]
})
print(f"User: Do you know anyone named Alice?")
print(f"Agent: {result_isolation['messages'][-1].content}")
print("\nBob's session should NOT know about Alice from the other session.")


Verifying session isolation:
User: Do you know anyone named Alice?
Agent: I don't have personal experiences or knowledge about specific individuals unless they are publicly known figures or fictional characters. If you're referring to a specific Alice, please provide more context!

Bob's session should NOT know about Alice from the other session.


## Cleanup

In [12]:
# Close the middleware to release Redis connections
await memory_middleware.aclose()
await memory_middleware_bob.aclose()
print("Middleware closed.")
print("Demo complete!")

Middleware closed.
Demo complete!
